#### Product Investigation:

This example demonstrates how a product owner can evaluate customer satisfaction from online reviews at scale. This process provides managers with higher fidelity information about the online market's sentiment for a product with specific justifications, allowing for more timely, accurate business decisions. 

Process: a product was selected at random from those containing over 75 reviews. Reviews were then segmented for investigation by sentiment analysis score. Exploring the review score's top and bottom tiers, clear insights emerge for actionable improvements. 

In [ ]:
import pandas as pd
import numpy as np

from pymongo import MongoClient

import re
import string
import pickle
import nltk
import spacy

from nltk.corpus import stopwords
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from gensim import corpora, models, similarities, matutils
from gensim.corpora import Dictionary
import gensim

from corextopic import corextopic as ct
from corextopic import vis_topic as vt
import scipy.sparse as ss

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

#### Data Import:

This data for this project were stored in MongoDB on AWS EC2 and accessed using the PyMongo module. The following code establishes connection, extracts a user-defined number of records from the electronics reviews collection, scans for null values, and creates a new dataframe for the columns of interest.

Star reviews are scaled to Net Promoter Scores with the following assumptions: 5 stars equates to a promoter, 4 stars is neutral, and 1-3 stars is a detractor.

In [ ]:
# connect to Mongo on AWS and pull data:

client = MongoClient('mongodb://username@1234567890:27017/amazon_reviews', password = '1234567890')

db = client.amazon_reviews
    
try:
    if db.list_collection_names() == ['electronics']:
        print("Connected to MongoDB on AWS EC2.\n")
    
    total_records = int(input('Specify number of records:'))
        
    # pull reviews from electronics category to load dataframe:

    elec = list(db.electronics.find().limit(total_records))
    df = pd.DataFrame(list(elec))
    print('>>', df.shape[0],'reviews retrieved from database\n')

except:
    print("Connection Error.")
    
check_null = df.isnull().values.any()
print('Null values in df =', check_null, '\n')

df = df[['product_id', 'product_title', 'review_body', 'review_headline', 'star_rating', 'total_votes', 'verified_purchase', 'vine']]

# Assign 5 star reviews to 1, 4 star reviews to 0, and 1-3 star reviews to -1:

df['nps'] = np.where(df['star_rating'] == 5, 1, np.where(df['star_rating'] == 4, 0, -1))

print('Unique products:', len(list(df.product_id.unique())),'\n')

df.to_pickle('df_records.pkl') 

df.head()

In [ ]:
# create dataframe of products with > 75 reviews:

df_counts = df.groupby(['product_id']).count().sort_values('product_title', ascending = False)
sort_mask = (df_counts['product_title'] > 75)

df_counts[sort_mask].head()

In [ ]:
# for this example, the 3rd product was "randomly" selected:

product_mask = (df['product_id'] == 'B00F5NE2KG')
df_product = df[product_mask]
df_product.head()

#### Exploratory Data Analysis:

As is often the case with online reviews, this product's ratings are skewed left with median equal to the max of 5 stars. As a derivative of star ratings, Net Promoter Score follows a similar pattern. With both medians also equal to the highest rating, we can hypothesize that there is signal loss in simplified ratings systems.

In [ ]:
df_product.describe()

In [ ]:
df_product.star_rating.hist()

print("Mean Star Rating:", df_product.star_rating.mean())
print("Median Star Rating:", df_product.star_rating.median())

In [ ]:
df_product.nps.hist()

print("Mean NPS Rating:", df_product.nps.mean())
print("Median NPS Rating:", df_product.nps.median())

#### Text Pre-processing:

The text data are processed with spaCy and loaded to a new column. The functions `alphanumeric` and `punc_lower` remove unwanted punctuation and convert text to lowercase, respectively.

In [ ]:
nlp = spacy.load('/home/ubuntu/anaconda3/lib/python3.7/site-packages/en_core_web_sm/en_core_web_sm-2.1.0/')

stop_words = spacy.lang.en.stop_words.STOP_WORDS
stop_words.update(['-PRON-', 'pron', 'br', '<br/>', '<br />', '<br>', '<br', 'br>', '</br', 'br/>', '<br /><br />', 'use', 'like', '/br', 'br/'])

# Prepare data to process with Spacy:

list_of_spacy = []

for document in nlp.pipe(df_product.review_body, n_threads = -1):
    try:
        list_of_spacy.append(document)
        
    except:
        list_of_spacy.append('bad_doc')
        print('bad doc:', document)
    
df_product['review_body_spacy'] = list_of_spacy

print(f'{len(list_of_spacy)} documents processed with Spacy\n')

# pre-process text with NLTK for Corex:

alphanumeric = lambda x: re.sub(r'<[^>]*>\w*\d\w*', '', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x.lower())

words = set(nltk.corpus.words.words())

nltk_list = []

df_product['cleaned_review_body_text'] = df_product['review_body_spacy'].apply\
(lambda x: ' '.join([ele.lemma_ for ele in x if ele.lemma_ not in stop_words and ele not in stop_words]))

for row in df_product['cleaned_review_body_text']:
    nltk_list.append(' '.join(w for w in nltk.wordpunct_tokenize(row) if w.lower() in words or not w.isalpha()))
    
df_product['nltk_terms'] = nltk_list

df_product['nltk_terms'] = df_product['nltk_terms'].map(alphanumeric).map(punc_lower)

df_product = df_product.drop(columns = ['cleaned_review_body_text'])

print('NLTK processing complete.')

df_product.head()

#### NLP Hypothesis Test:

Before delving into a protracted NLP analysis to find fidelity in language where the ratings have disappointed, it is important to determine whether a general correlation between product reviews and ratings exists. 

This will be accomplished by applying classification to a Bag-of-Words model, specifically using CountVectorizer and Logistic Regression. 

In [ ]:
# create a new dataframe with review body text and nps:

df2 = df[['review_body', 'nps']]
df2 = df2.drop(df2[df2.nps == 0].index) # remove neutral rating (4 stars) for binary classification
df2.head()

In [ ]:
# define features & labels:

X = df2.review_body.values.astype('unicode')
y = df2.nps

# split into training / test sets:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# The first document-term matrix has default Count Vectorizer values - counts of unigrams:

cv1 = CountVectorizer(max_features=20000, stop_words='english', token_pattern="\\b[a-z][a-z]+\\b", binary=True)

X_train_cv1 = cv1.fit_transform(X_train)
X_test_cv1  = cv1.transform(X_test)

pd.DataFrame(X_train_cv1.toarray(), columns=cv1.get_feature_names()).head()

In [ ]:
# Create a second document-term matrix has both unigrams and bigrams, and indicators instead of counts:

cv2 = CountVectorizer(max_features=20000, stop_words='english', ngram_range=(1,2), token_pattern="\\b[a-z][a-z]+\\b", binary=True)
                    
X_train_cv2 = cv2.fit_transform(X_train)
X_test_cv2  = cv2.transform(X_test)

pd.DataFrame(X_train_cv2.toarray(), columns=cv2.get_feature_names()).head()

In [ ]:
# Create a logistic regression model to use
lr = LogisticRegression()

# Train the first model
lr.fit(X_train_cv1, y_train)
y_pred_cv1 = lr.predict(X_test_cv1)


# Train the second model
lr.fit(X_train_cv2, y_train)
y_pred_cv2 = lr.predict(X_test_cv2)

In [ ]:
# Create a function to calculate the error metrics, since we'll be doing this several times

def conf_matrix(actual, predicted):
    cm = confusion_matrix(actual, predicted)
    sns.heatmap(cm, xticklabels=['predicted_negative', 'predicted_positive'], 
                yticklabels=['actual_negative', 'actual_positive'], annot=True,
                fmt='d', annot_kws={'fontsize':20}, cmap="YlGnBu");

    true_neg, false_pos = cm[0]
    false_neg, true_pos = cm[1]

    accuracy = round((true_pos + true_neg) / (true_pos + true_neg + false_pos + false_neg),3)
    precision = round((true_pos) / (true_pos + false_pos),3)
    recall = round((true_pos) / (true_pos + false_neg),3)
    f1 = round(2 * (precision * recall) / (precision + recall),3)

    cm_results = [accuracy, precision, recall, f1]
    return cm_results

#### Results:

The supervised learning trial confirms a strong connection between reviews and ratings with all metrics above 90% and F1 scores nearing 94%.

In [ ]:
# Heat map for the first logistic regression model:

cm1 = conf_matrix(y_test, y_pred_cv1)

In [ ]:
# Heat map for the second logistic regression model

cm2 = conf_matrix(y_test, y_pred_cv2)

In [ ]:
# Compare results:

results = pd.DataFrame(list(zip(cm1, cm2)))
results = results.set_index([['Accuracy', 'Precision', 'Recall', 'F1 Score']])
results.columns = ['LogReg1', 'LogReg2']
results

#### VADER Sentiment Analysis:

The star ratings & NPS data's extreme left skew exhibits a bias for 5 Star Ratings. How then does a product discern between those 5 Star Ratings from lethargic or generous customers and those reviews indicative of raving fans?

The most enthusiastic fans were isolated from the "5-Star by Default" population using Valence Aware Dictionary and sEntiment Reasoner (VADER) Sentiment Analysis. A filter coupling VADER scores over 0.95 with an NPS Score of 1 (as a checksum for sarcasm) was then used to populate a dataframe comprised of our most satisfied customers. This process was then repeated with inverse parameters to isolate the least satisfied customers.

In [ ]:
# use VADER with original reviews column as input to include emotionally rich punctuation and capitalization: 

analyzer = SentimentIntensityAnalyzer()

# compound is the normalized, weighted composite score:

df_product['vader'] = [analyzer.polarity_scores(row)['compound'] for row in df_product.review_body] 

print(f'created VADER scores for {df_product.vader.shape[0]} rows\n')

df_product.head()

In [ ]:
df_product.vader.hist()

In [ ]:
df_product.corr()

In [ ]:
# explore top reviews using a vader filter > 0.95:

df_p1_top_mask = (df_product['vader'] > 0.95)

df_product[df_p1_top_mask].nps.hist()

In [ ]:
df_product[df_p1_top_mask].star_rating.hist()

In [ ]:
# use a combination of vader and nps to create dataframe for top and bottom reviews:

df_p1_top_mask = (df_product['vader'] > 0.95) & (df_product['nps'] == 1)
df_top = df_product[df_p1_top_mask]

df_p1_bot_mask = (df_product['vader'] < -0.25) & (df_product['nps'] == -1)
df_bot = df_product[df_p1_bot_mask]


In [ ]:
cor_vectorizer = CountVectorizer(max_features=20000, ngram_range=(1,2), binary=True, token_pattern="\\b[a-z][a-z]+\\b", stop_words='english')

cor_doc_word_top = cor_vectorizer.fit_transform(df_top['nltk_terms'])
cor_words = list(np.asarray(cor_vectorizer.get_feature_names())) 
topic_model_top = ct.Corex(n_hidden=6, words = cor_words, seed=1)
topic_model_top.fit(cor_doc_word_top, words = cor_words, docs = df_top.nltk_terms)

# repeat process for bottom reveiws:

cor_doc_word_bot = cor_vectorizer.fit_transform(df_bot['nltk_terms'])
cor_words = list(np.asarray(cor_vectorizer.get_feature_names()))
topic_model_bot = ct.Corex(n_hidden=6, words = cor_words, seed=1) # must be repeated
topic_model_bot.fit(cor_doc_word_bot, words = cor_words, docs = df_bot.nltk_terms)


#### Topic Evaluation: Top Reviews

CorEx topic models operate on the principle of mutual information between topic words and the topic - not the highest probability of a topic word occurring. 

In [ ]:
# Print all topics from the top topic model:

topics = topic_model_top.get_topics()
for n, topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

In [ ]:
# retrieve original reviews & top topics for parameters specified:

def TopicReviews(topic_number, topic_model_version, topic_dataframe):
    
    print('Topics: \n\n', topic_model_version.get_topics()[topic_number])

    top_docs = topic_model_version.get_top_docs(topic=topic_number, n_docs=3)

    for i in top_docs:
        temp = (topic_dataframe.loc[topic_dataframe['nltk_terms'] == i[0]]).index[0]  # get row index of review for original review
        print('\nIndex:', temp)
        print('Rating:', df.star_rating.iloc[temp])
        print('Review:', df.review_body.iloc[temp])  # print original review

#### Topic 0: 

In Topic 0, we can see that the word "quality" has the highest mutual information and sets the general topic theme. From Topic 0's other top words, we can find other specific justifications for why customers wrote enthusiastically positive reviews: 

* convenient
* simple
* works in wet environments (sauna)

CorEx also allows us to `get_top_docs` to read the reviews most highly correlated with a topic. Since the text processing makes the CorEx input reviews difficult to read, I've indexed the original reviews for legibility.

The top 3 reviews give us a better idea of what is meant by the "quality" topic: sound. Customers also like the size and portability.
From a marketing perspective, these reviews that best define our quality topic tell us that customers are purchasing the product for their kids and as gifts.

In [ ]:
TopicReviews(0, topic_model_top, df_top)

#### Topic 1:

Topic 1's general theme is **exceeds expectations**. Not only is "*expect*" the top topic words, but the top documents show a relationship between its "smaller than expected" size producing "more than we expected for sound quality." This is all given, "I did not expect much given its moderate price." For a product named "MagicBox," exceeding expectations is brand-consistent.

Topic 1 also reinforces insights from Topic 0: customers are buying the speaker for their children, giving it as gifts, and using it in wet environments.

In [ ]:
TopicReviews(1, topic_model_top, df_top)

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(range(topic_model_top.tcs.shape[0]), topic_model_top.tcs, color='#ffa500', width=0.5)
plt.title('Correlation by Topic: Top Reviews')
plt.xlabel('Topic', fontsize=16)
plt.ylabel('Total Correlation (nats)', fontsize=16);

#### Topic Evaluation: Bottom Reviews


In [ ]:
# Print all topics from the bottom topic model:

topics = topic_model_bot.get_topics()
for n, topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

#### Topic 0:

This topic words show clustering around sound issues when using Bluetooth:

* high frequency noise
* audio quality
* pairing & range

These findings are surprising considering that most reviews are positive and mention superb sound quality. Further, 2 of the top documents mention the sound quality improving when operating from an auxillary cable. 

This topic reveals a potential manufacturing or component problem involving Bluetooth that is detracting from product performance.

In [ ]:
TopicReviews(0, topic_model_bot, df_bot)

#### Topic 1:

Topic 1 shares the defective product theme with Topic 0 and it follows that a top document is also repeated. However, this topic offers more information as to how the product is failing: 

* broken charge port
* defective volume controls

Since this product was sold online and reviews can be mapped to customers, the management has the ability to investigate supply chain and / or manufacturing patterns causing this defective lot.

In [ ]:
TopicReviews(1, topic_model_bot, df_bot)

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(range(topic_model_bot.tcs.shape[0]), topic_model_bot.tcs, color='#4e79a7', width=0.5)
plt.title('Correlation by Topic: Bottom Reviews')
plt.xlabel('Topic', fontsize=16)
plt.ylabel('Total Correlation (nats)', fontsize=16);